## Part 02: Export nodes and select working streets
michael babb  
2024 11 24

In [1]:
# standard
import os

In [2]:
# external
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely

In [3]:
# custom
import run_constants as rc
from utils import generate_street_end_vertices, write_gdf

# load the streetnetwork

In [4]:
fpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S02_SND_IN_FILE_NAME)

In [6]:
gdf = gpd.read_file(filename = fpn)

In [7]:
# update the city portion variable

In [8]:
# after performing some analysis in qGIS, we can quickly identify the central streets
fpn = os.path.join(rc.INPUT_FILE_PATH, rc.S02_CENTRAL_STREETS_IN_FILE_NAME)
cs_df = pd.read_csv(filepath_or_buffer=fpn)


In [9]:
cs_df.head()

,snd_id,temp_city_group
0,21082,CNTR
1,25880,CNTR
2,16760,CNTR
3,20437,CNTR
4,21232,CNTR


In [10]:
test_join = pd.merge(left = gdf, right = cs_df, how = 'left', indicator = True)
test_join.shape
test_join['_merge'].value_counts()

_merge
left_only     32667
both           1711
right_only        0
Name: count, dtype: int64

In [11]:
test_join.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_portion', 'geometry',
       'temp_city_group', '_merge'],
      dtype='object')

In [12]:
test_join.loc[test_join['_merge'] == 'both', 'city_portion'] = \
test_join.loc[test_join['_merge'] == 'both', 'temp_city_group']

In [13]:
write_gdf(gdf = test_join, output_file_path=rc.OUTPUT_FILE_PATH,
          output_file_name='test_city_portion_update.gpkg')

In [15]:
drop_col_names = ['temp_city_group', '_merge']
gdf = test_join.drop(labels = drop_col_names, axis = 1).copy()

# generate street end vertices - these will be helpful with identification in subsequent steps

In [16]:
# can we generate the vertices? yes... but let's check what type of geometry we are working with
gdf['geometry'].map(lambda x: x.geom_type).value_counts()

geometry
MultiLineString    34378
Name: count, dtype: int64

In [17]:
# everything is a MultLineString??? is that necessary?
# we can call the gdf.explode() function to convert MultiLineStrings to LineStrings
# if the record counts are the same, we can convert everything to LineStrings 

In [18]:
test_gdf = gdf.explode()

In [19]:
test_gdf.shape

(34378, 38)

In [20]:
# nope! The number of records is the same, indicating that MultiLineStrings are not necessary

In [21]:
gdf = test_gdf.copy()

In [22]:
gdf['geometry'].geom_type.value_counts()

LineString    34378
Name: count, dtype: int64

In [23]:
gdf, node_gdf = generate_street_end_vertices(gdf = gdf)

In [24]:
write_gdf(gdf = node_gdf, output_file_path=rc.OUTPUT_FILE_PATH,
          output_file_name= rc.S02_NODE_OUT_FILE_NAME)

# import the manually classified streets to update streets without a classification. 

In [25]:
fpn = os.path.join(rc.INPUT_FILE_PATH, rc.S02_BST_IN_FILE_NAME)

In [26]:
blank_street_type_df = pd.read_excel(io = fpn)

In [27]:
blank_street_type_df.head()

,ord_stname_concat,ord_street_type_fix
0,NE SUNRISE VISTA,ST
1,NW ESPLANADE,ST
2,BROADWAY E,ST
3,FAUNTLEE CREST SW,ST
4,NE FOREST VISTA,ST


In [28]:
gdf = pd.merge(left = gdf, right = blank_street_type_df, how = 'left')

In [29]:
gdf.loc[gdf['ord_street_type'].isna(), 'ord_street_type'] = gdf.loc[gdf['ord_street_type'].isna(), 'ord_street_type_fix']

In [30]:
gdf['ord_street_type'].unique()

array(['AVE', 'CT', 'ST', 'PL', 'LN', 'RD', 'DR', 'WAY', 'PKWY', 'BLVD',
       'CIR', 'TER', 'AL', 'RP', 'LOOP', 'SR', 'IS', 'OP', 'TRL', 'RR',
       'BR', 'WKWY', 'ET', 'STCR', 'VIEW', 'FLYOVER', 'HWY', 'RN', 'PZ',
       'MALL'], dtype=object)

In [31]:
gdf = gdf.drop(labels = ['ord_street_type_fix'], axis = 1)

# keep only streets in Seattle

In [32]:
gdf = gdf.loc[(gdf['l_city'] == 'SEATTLE') |
(gdf['r_city'] == 'SEATTLE'), :].copy()

In [35]:
gdf.shape

(27891, 38)

# write the full seattle streets to disk

In [33]:
write_gdf(gdf = gdf, output_file_path=rc.OUTPUT_FILE_PATH,
          output_file_name=rc.S02_SND_FULL_OUT_FILE_NAME)

# export the individual street portions to make the different sections

In [38]:
# working_gdf
# winnow down the list of streets to get what we're looking for
q_str = 'segment_type in (1, )'
wgdf = gdf.query(expr = q_str)

# draw a convex hull around each group of city streets

In [39]:
data_list = []
geom_list = []
for cp in wgdf['city_portion'].unique().tolist():
    print(cp)
    temp_geoms = wgdf.loc[wgdf['city_portion'] == cp, 'geometry']
    
    # create the convex hull
    geoms = shapely.unary_union(geometries = temp_geoms)
    cvx_hull = shapely.convex_hull(geometry = geoms)
    temp_list = [cp, 'convex']
    data_list.append(temp_list)
    geom_list.append(cvx_hull)

S
E
NW
N
CNTR
NE
SW
W


In [40]:
output_gdf = gpd.GeoDataFrame(data = data_list, columns = ['city_portion', 'hull_type'], geometry = geom_list, crs = 'epsg:4326')

In [41]:
write_gdf(gdf = output_gdf, output_file_path=rc.OUTPUT_FILE_PATH,
          output_file_name=rc.S02_CITY_SECTIONS_OUT_FILE_NAME)

# Keep only streets with segment_type == 1, these are actual streets.
Everything else not a street (rail, trail, staircase, etc...).  
`segment_type == 15` is a stub, a dangling piece of a street, usually around 50 feet or less.  
Very short! There are 362 of them.

In [ ]:
gdf['segment_type'].value_counts()

In [ ]:
# before...
gdf.shape

In [39]:
gdf = gdf.loc[gdf['segment_type'] == 1, :].copy()

In [ ]:
# after...
gdf.shape

# remove the following streets types:

AL: alley  
TRL: trail  
OP: overpass  
IS: interstate  
SR: state route  
RR: rail  
FLYOVER: flyover  
STCR: streetcar  
ET: extension  
RN: turn  
RP: highway ramps  
WKWY: walkways  

In [41]:
# remove streets of the following type
street_type_to_remove = [ 'AL', 'TRL', 'OP', 'IS', 'SR', 'RR', 'FLYOVER', 'STCR', 'ET', 'RN', 'RP', 'WKWY']

In [ ]:
# before...
gdf.shape

In [ ]:
# what street types are left?
gdf['ord_street_type'].unique()

In [44]:
# remove...
gdf = gdf.loc[-gdf['ord_street_type'].isin(street_type_to_remove), :].copy()

In [ ]:
# after..
gdf.shape

In [ ]:
# what street types are left?
gdf['ord_street_type'].unique()

# remove other short segments that appear to be invalid.

In [48]:
# uncomment to remove the erroneous segments
err_segments = pd.read_csv(filepath_or_buffer='../data/streets_to_remove.txt')


In [ ]:
# examine the erroneous segments
tgdf = gdf.loc[gdf['snd_id'].isin(err_segments['snd_id']), :].copy()
tgdf['gis_seg_length'].describe()
# very short!

In [ ]:
# check names...
tgdf['ord_stname_concat'].unique().tolist()

In [ ]:
tgdf['segment_type'].unique()

In [ ]:
tgdf.head()

In [53]:
# remove the erroneous segments
gdf = gdf.loc[-gdf['snd_id'].isin(err_segments['snd_id']), :].copy()

In [ ]:
# after
gdf.shape

In [ ]:
# unique roads
gdf['ord_stname_concat'].unique().shape

In [ ]:
# total miles
gdf['gis_seg_length'].sum() / 5280

# save as a gdf - overwrite

In [57]:
write_gdf(gdf = gdf, output_file_path = rc.OUTPUT_FILE_PATH,
          output_file_name =  rc.S02_SND_WORKING_OUT_FILE_NAME)